# Propose-and-Reject Algorithm example
Example and text was adapted from CS 70 Spring 2020 Note 5 - Stable Matching (http://www.eecs70.org/static/notes/n5.pdf)

This notebook explores the propose-and-reject algorithm from the Stable Matching module of CS 70. It shows the algorithm running in code. The notebook will show the ways in which Theorems, Lemmas and Observations from Note 5 are reflected in code. 

The utils.py contains the Job and Candidate class defined, feel free to read through the code to understand the different methods and attributes of each class. We will be making extensive use of the two classes and their methods in running the Propose-and-reject algorithm. 

In [127]:
# Implementation of Job class. 
class Job(object):
    def __init__(self, name):
        """Initializes the job with name, preferences, current offer and rejected_from attributes."""
        self.name = name
        self.preferences = []
        self.currently_offered_to = ""
        self.rejected_from = []
    
    def add_preference(self, choices):
        """This method adds the preferences in order (increasing to decreasing) for each job."""
        for choice in choices:
            self.preferences.append(choice)
        
    def set_preferences_as_iterator(self):
        """This creates an iterator to allow getting the next preference on demand."""
        self.preferences = iter(self.preferences)
        
    def obtain_rejected(self):
        """Returns the list of candidates that have rejected this job."""
        return self.rejected_from
    
    def obtain_name(self):
        """Returns the name of the job."""
        return self.name
        
    
    def offer_job(self):
        """Proposes to the next available candidate on the Job's preference list"""
        if self.currently_offered_to == "":
            self.currently_offered_to = next(self.preferences)
            self.currently_offered_to.offer(self)
        elif self.currently_offered_to in self.rejected_from:
            self.currently_offered_to = next(self.preferences)
            self.currently_offered_to.offer(self)

In [128]:
# Candidate class
class Candidate(object):
    def __init__(self, name):
        """Initializes the job with name, preferences, current offered, currently_on_string
        and rejected_from attributes."""
        self.name = name
        self.preferences = []
        self.currently_offered = []
        self.currently_on_string = ""
        self.rejected = self.preferences
    
    def name(self):
        """Returns the name of the Candidate."""
        return self.name
        
    def add_preference(self, choices):
        """Adds the preferences in decreasing-order-of-preference to self.preferences"""
        for choice in choices:
            self.preferences.append(choice)
        
    def offer(self, job):
        """Receives the offer from a job for this particular candidate."""
        self.currently_offered.append(job)
    
    def reject(self, job):
        """Rejects a particular job in the afternoon."""
        self.currently_offered.remove(job)
        job.obtain_rejected().append(self)
        
    def obtain_rejected(self):
        """Returns the list of all the jobs that the candidate rejected."""
        return self.rejected
    
    def obtain_currently_offered(self):
        """Returns all offers received for the day"""
        return self.currently_offered
    
    def obtain_currently_on_string(self):
        """Retursn the job that is currently on-string for the candidate."""
        return self.currently_on_string
        
    def pick(self):
        """Chooses the best offer received by the candidate and rejects the rest."""
        if len(self.currently_offered) != 0:
            self.currently_on_string = min(self.currently_offered, key=lambda x: self.preferences.index(x))
            self.rejected = [a for a in self.currently_offered if a != self.currently_on_string]
            for job in self.rejected:
                self.reject(job)

In [129]:
# Other Utilities

# Adds all the preferences for all jobs and candidates to their preference list by calling the 
# add_preference method
def add_all_preferences(preferences_dic):
    for (key, value) in preferences_dic.items():
        key.add_preference(preferences_dic[key])

# Propose-and-Reject algorithm
def propose_reject(all_candidates, all_jobs):
    day_counter = 1
    
    # The propose-and-reject algorithm will continue running until there are no rejections
    while len([a for a in all_candidates if len(a.obtain_rejected()) != 0]) != 0:
        
        # prints day's number
        print("Day: ", day_counter)

        # In the morning all jobs send an offer to their top candidates. 
        for job in all_jobs:
            job.offer_job()

        print("Afternoon:")
        print()
        for candidate in all_candidates:
            print("All offers received for " + candidate.name + ": ", 
                  [a.obtain_name() for a in candidate.obtain_currently_offered()])

        # In the afternoon and evening, the candidates puts their best offer on string and reject the rest.
        for candidate in all_candidates:
            candidate.pick()

        # Increments the counter for days by 1, as the day ends. 
        day_counter += 1

        print()
        print("Evening")
        print()
        for candidate in all_candidates:
            if candidate.obtain_currently_on_string() == "":
                print("Currently on string for ", candidate.name, ": None")
            else:
                print("Currently on string for ", candidate.name, ": ", 
                      candidate.obtain_currently_on_string().obtain_name())
        print("-------------------")

So, the first few pages of the notes talk about the following example, where we have three companies (Approximation_Inc, Basis_Co., Control Corp.) proposing to three candiates (Anita, Bridget, Christine) by the propose-and-reject algorithm. 

The following table lists the preferences of each company i.e. the order in which they prefer the candidates (Table 1).


<table>
    <tr>
        <th> Company Name </th>
        <th> Preference 1 </th>
        <th> Preference 2 </th>
        <th> Preference 3 </th>
    </tr>
    <tr>
        <td> Approximation Inc. </td>
        <td> Anita </td> 
        <td> Bridget </td>
        <td> Christine </td>
    </tr>
    <tr>
        <td> Basis Co. </td>
        <td> Bridget </td> 
        <td> Anita </td>
        <td> Christine </td>
    </tr>
    <tr>
        <td> Control Corp. </td>
        <td> Anita </td> 
        <td> Bridget </td>
        <td> Christine </td>
    </tr>
</table>
<br>
<br>
The following table shows the preferences of each candidate - i.e. the order in which they prefer each of the jobs (Table 2). 
<table>
    <tr>
        <th> Candidate Name </th>
        <th> Preference 1 </th>
        <th> Preference 2 </th>
        <th> Preference 3 </th>
    </tr>
    <tr>
        <td> Anita </td>
        <td> Basis Co. </td> 
        <td> Approximation Inc. </td>
        <td> Control Corp. </td>
    </tr>
    <tr>
        <td> Bridget </td>
        <td> Approximation Inc. </td> 
        <td> Basis Co. </td>
        <td> Control Corp. </td>
    </tr>
    <tr>
        <td> Christine </td>
        <td> Approximation Inc. </td> 
        <td> Basis Co. </td>
        <td> Control Corp. </td>
    </tr>
</table>
<br>
<br>

Now, we begin the implementation of the Propose-and-reject algorithm. If you feel you need to review the specifics of the algorithm, feel free to use the diagram below, which explains the steps of the algorithm. 

<figure>
    <img src="70_P&R_1.png">
    <figcaption>Fig.1 - Propose-and-Reject Algorithm Steps (Adapted from CS 70 Note 5 - Stable Matching)</figcaption>
</figure>

<br>
<br>
The cell below does 3 things. <br>
Step 1: Initializes 3 jobs and 3 candidates. <br>
Step 2: Adds the preferences for each job and candidate. <br>
Step 3: Runs the propose-and-reject algorithm. 

In [130]:
# Initializing all jobs
approximation_inc = Job("Approximation Inc.")
basis_co = Job("Basis Co.")
control_corp = Job("Control Corp.")
all_jobs = [approximation_inc, basis_co,  control_corp]

# Initializing all candidates
anita = Candidate("Anita")
bridget = Candidate("Bridget")
christine = Candidate("Christine")
all_candidates = [anita,  bridget, christine]

# Orders the preferences for each job and candidate, by storing the preferences in a dictionary
all_job_preferences_dic = {approximation_inc: [anita, bridget, christine], 
                       basis_co:[bridget, anita, christine], 
                       control_corp:[anita, bridget, christine]}

all_candidate_preferences_dic = {anita: [basis_co, approximation_inc, control_corp],
                                 bridget: [approximation_inc, basis_co, control_corp],
                                 christine: [approximation_inc, basis_co, control_corp]}

# Adds all preferences
add_all_preferences(all_job_preferences_dic)
add_all_preferences(all_candidate_preferences_dic)

# Converts the preference list of all jobs to be iterators to obtain the next on the list on-demand        
for i in all_jobs:
    i.set_preferences_as_iterator()

# Running Propose and Reject
propose_reject(all_candidates, all_jobs)

Day:  1
Afternoon:

All offers received for Anita:  ['Approximation Inc.', 'Control Corp.']
All offers received for Bridget:  ['Basis Co.']
All offers received for Christine:  []

Evening

Currently on string for  Anita :  Approximation Inc.
Currently on string for  Bridget :  Basis Co.
Currently on string for  Christine : None
-------------------
Day:  2
Afternoon:

All offers received for Anita:  ['Approximation Inc.']
All offers received for Bridget:  ['Basis Co.', 'Control Corp.']
All offers received for Christine:  []

Evening

Currently on string for  Anita :  Approximation Inc.
Currently on string for  Bridget :  Basis Co.
Currently on string for  Christine : None
-------------------
Day:  3
Afternoon:

All offers received for Anita:  ['Approximation Inc.']
All offers received for Bridget:  ['Basis Co.']
All offers received for Christine:  ['Control Corp.']

Evening

Currently on string for  Anita :  Approximation Inc.
Currently on string for  Bridget :  Basis Co.
Currently on s

## Optimality

The cell below shows how the propose-and-reject algorithm is always job-optimal.

In [131]:
# Initializing all jobs
approximation_inc = Job("Approximation Inc.")
basis_co = Job("Basis Co.")
control_corp = Job("Control Corp.")
dmean_d = Job("Dmean D.")
all_jobs = [approximation_inc, basis_co,  control_corp, dmean_d]

# Initializing all candidates
anita = Candidate("Anita")
bridget = Candidate("Bridget")
christine = Candidate("Christine")
duffer = Candidate("Duffer")
all_candidates = [anita,  bridget, christine,duffer]

# Adding preferences
all_job_preferences_dic = {approximation_inc: [anita, bridget, christine, duffer], 
                           basis_co:[anita, duffer, christine, bridget], 
                           control_corp:[anita,christine, bridget, duffer],
                           dmean_d: [anita, bridget, christine, duffer]
                          }

all_candidate_preferences_dic = {anita: [approximation_inc, control_corp, basis_co, dmean_d],
                                 bridget: [dmean_d, control_corp, basis_co, approximation_inc],
                                 christine: [basis_co, control_corp, approximation_inc, dmean_d],
                                 duffer: [control_corp, dmean_d, basis_co, approximation_inc]}

# Adds all preferences
add_all_preferences(all_job_preferences_dic)
add_all_preferences(all_candidate_preferences_dic)

# Converts the preference list of all jobs to be iterators to obtain the next on the list on-demand        
for i in all_jobs:
    i.set_preferences_as_iterator()

# Running Propose and Reject
propose_reject(all_candidates, all_jobs)

Day:  1
Afternoon:

All offers received for Anita:  ['Approximation Inc.', 'Basis Co.', 'Control Corp.', 'Dmean D.']
All offers received for Bridget:  []
All offers received for Christine:  []
All offers received for Duffer:  []

Evening

Currently on string for  Anita :  Approximation Inc.
Currently on string for  Bridget : None
Currently on string for  Christine : None
Currently on string for  Duffer : None
-------------------
Day:  2
Afternoon:

All offers received for Anita:  ['Approximation Inc.']
All offers received for Bridget:  ['Dmean D.']
All offers received for Christine:  ['Control Corp.']
All offers received for Duffer:  ['Basis Co.']

Evening

Currently on string for  Anita :  Approximation Inc.
Currently on string for  Bridget :  Dmean D.
Currently on string for  Christine :  Control Corp.
Currently on string for  Duffer :  Basis Co.
-------------------


The cell below shows how if we make the candidates propose to the jobs, the propose-and-reject algorithm will be candidate-optimal. So, all the "jobs" will have the same name as the candidates and all the "candidates" will have the same name as all the jobs. 

Treat "jobs" like proposing entities and treat "candidates" like being proposed to entities.

In [132]:
# Initializing all jobs
anita = Job("Anita")
bridget = Job("Bridget")
christine = Job("Christine")
duffer = Job("Duffer")

# Initializing all candidates
approximation_inc = Candidate("Approximation Inc.")
basis_co = Candidate("Basis Co.")
control_corp = Candidate("Control Corp.")
dmean_d = Candidate("Dmean D.")
all_jobs = [anita,  bridget, christine,duffer]
all_candidates = [approximation_inc, basis_co,  control_corp, dmean_d]

# Adding preferences
all_job_preferences_dic = {approximation_inc: [anita, bridget, christine, duffer], 
                           basis_co:[anita, duffer, christine, bridget], 
                           control_corp:[anita,christine, bridget, duffer],
                           dmean_d: [anita, bridget, christine, duffer]
                          }

all_candidate_preferences_dic = {anita: [approximation_inc, control_corp, basis_co, dmean_d],
                                 bridget: [dmean_d, control_corp, basis_co, approximation_inc],
                                 christine: [basis_co, control_corp, approximation_inc, dmean_d],
                                 duffer: [control_corp, dmean_d, basis_co, approximation_inc]}
# Adds all preferences
add_all_preferences(all_job_preferences_dic)
add_all_preferences(all_candidate_preferences_dic)

# Converts the preference list of all jobs to be iterators to obtain the next on the list on-demand        
for i in all_jobs:
    i.set_preferences_as_iterator()

# Running Propose and Reject
propose_reject(all_candidates, all_jobs)

Day:  1
Afternoon:

All offers received for Approximation Inc.:  ['Anita']
All offers received for Basis Co.:  ['Christine']
All offers received for Control Corp.:  ['Duffer']
All offers received for Dmean D.:  ['Bridget']

Evening

Currently on string for  Approximation Inc. :  Anita
Currently on string for  Basis Co. :  Christine
Currently on string for  Control Corp. :  Duffer
Currently on string for  Dmean D. :  Bridget
-------------------


#### Contributors:
<ul>
    <li> Priyans Nishithkumar Desai </li>
</ul>